In [4]:
import os
import time
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import tqdm
import gc
from textwrap import dedent

# ======================================
# ✅ 환경 설정
# ======================================

load_dotenv(dotenv_path="C:/Aicamp/SKN13_my/13_Langchain/.env")

PERSIST_DIR = "vector_store/vitamin_chroma"
os.makedirs(PERSIST_DIR, exist_ok=True)

EMBEDDING_MODEL = OpenAIEmbeddings(model="text-embedding-3-large")

# ======================================
# ✅ 유틸리티 함수
# ======================================

def safe_request_get(url, params=None, headers=None, retries=3, backoff=1, timeout=5):
    """
    안전한 GET 요청 함수 (retry 및 backoff 포함).
    """
    for attempt in range(retries):
        try:
            res = requests.get(url, params=params, headers=headers, timeout=timeout)
            res.raise_for_status()
            return res
        except requests.exceptions.RequestException as e:
            print(f"⏳ 요청 실패 ({attempt + 1}/{retries}): {e}")
            time.sleep(backoff * (attempt + 1))
    raise Exception(f"❌ 요청 실패: {url}")

def get_existing_pmcids(collection_name, persist_dir):
    """
    이미 저장된 PMCID 목록을 가져옵니다.
    """
    vector_store = Chroma(
        collection_name=collection_name,
        embedding_function=EMBEDDING_MODEL,
        persist_directory=persist_dir
    )
    metadatas = vector_store.get().get("metadatas", [])
    return {m.get("pmcid") for m in metadatas if m and m.get("pmcid")}

def extract_abstract_html(pmcid):
    """
    PMCID 페이지에서 abstract HTML을 추출합니다.
    """
    url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmcid}/"
    headers = {"User-Agent": "Mozilla/5.0"}
    res = safe_request_get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    abstract_section = soup.find("section", class_="abstract")
    if not abstract_section:
        print(f"⚠️ abstract section 없음 (PMCID: {pmcid}) → 건너뜀")
        return None

    paragraphs = abstract_section.find_all("p")
    abstract_text = "\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))
    if not abstract_text:
        print(f"⚠️ abstract 내용 없음 (PMCID: {pmcid}) → 건너뜀")
        return None

    return abstract_text

def chunk_text_to_documents(text, r, url):
    """
    abstract를 chunk 단위 Document로 분할합니다.
    """
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = splitter.create_documents([text])
    metadata = {
        "pmcid": r.get("pmcid", ""),
        "title": r.get("title", ""),
        "doi": r.get("doi", ""),
        "pubYear": r.get("pubYear", ""),
        "journal": r.get("journalTitle", ""),
        "authors": r.get("authorString", ""),
        "source": url
    }
    for doc in docs:
        doc.metadata = metadata
    return docs

def store_documents_batch(docs, collection_name, persist_dir, batch_size=5):
    """
    Vector Store에 문서를 batch 단위로 저장합니다.
    """
    vector_store = Chroma(
        collection_name=collection_name,
        embedding_function=EMBEDDING_MODEL,
        persist_directory=persist_dir
    )
    for i in range(0, len(docs), batch_size):
        chunk = docs[i:i+batch_size]
        vector_store.add_documents(chunk)

# ======================================
# ✅ 크롤링 + 저장
# ======================================

def process_topic(query, collection_name, persist_dir, batch_size):
    cursor = "*"
    page_size = 5

    existing_pmcs = get_existing_pmcids(collection_name, persist_dir)

    init_res = safe_request_get(
        "https://www.ebi.ac.uk/europepmc/webservices/rest/search",
        params={"query": query, "format": "json", "pageSize": 1},
        headers={"User-Agent": "Mozilla/5.0"}
    )
    init_data = init_res.json()
    total_count = init_data.get("hitCount", 0)
    print(f"🔎 {collection_name}: EuropePMC 검색 결과 개수: {total_count}개")

    if total_count == 0:
        print(f"⚠️ {collection_name}: 검색 결과 없음 → 패스")
        return

    all_new_docs = []
    new_pmcs = set()
    progress_index = 1
    pbar = tqdm.tqdm(desc=f"{collection_name} 진행", unit="논문", total=total_count)

    try:
        while True:
            res = safe_request_get(
                "https://www.ebi.ac.uk/europepmc/webservices/rest/search",
                params={"query": query, "format": "json", "pageSize": page_size, "cursorMark": cursor},
                headers={"User-Agent": "Mozilla/5.0"}
            )
            data = res.json()
            results = data.get("resultList", {}).get("result", [])
            if not results:
                break

            for r in results:
                pmcid = r.get("pmcid", "")

                if not pmcid:
                    print(f"[{progress_index}/{total_count}] ⚠️ PMCID 없음 → 건너뜀")
                    progress_index += 1
                    pbar.update(1)
                    continue

                if pmcid in existing_pmcs or pmcid in new_pmcs:
                    print(f"[{progress_index}/{total_count}] ⚠️ 이미 저장된 논문 건너뜀 (PMCID: {pmcid})")
                    progress_index += 1
                    pbar.update(1)
                    continue

                abstract_text = extract_abstract_html(pmcid)
                if not abstract_text:
                    progress_index += 1
                    pbar.update(1)
                    continue

                url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmcid}/"
                docs = chunk_text_to_documents(abstract_text, r, url)
                all_new_docs.extend(docs)
                new_pmcs.add(pmcid)

                print(f"[{progress_index}/{total_count}] ✅ 저장 완료 (PMCID: {pmcid})")

                if len(all_new_docs) >= batch_size:
                    store_documents_batch(all_new_docs, collection_name, persist_dir, batch_size)
                    all_new_docs = []
                    gc.collect()

                progress_index += 1
                pbar.update(1)

            prev_cursor = cursor
            cursor = data.get("nextCursorMark")
            if not cursor or cursor == prev_cursor:
                break

    except KeyboardInterrupt:
        print("\n🛑 중단 감지! 지금까지의 데이터를 저장합니다...")
        if all_new_docs:
            store_documents_batch(all_new_docs, collection_name, persist_dir, batch_size)
            print(f"⚡ 중단 시점까지 {len(new_pmcs)}개 논문 저장 완료")
        else:
            print("⚡ 저장할 새 문서 없음")
        pbar.close()
        gc.collect()
        raise

    if all_new_docs:
        store_documents_batch(all_new_docs, collection_name, persist_dir, batch_size)
        print(f"\n총 {len(new_pmcs)}개 논문 {collection_name}에 저장 완료 (batch size: {batch_size})")
    else:
        print("\n저장할 새 논문이 없습니다.")

    pbar.close()
    gc.collect()

# ======================================
# ✅ 실행 예제
# ======================================

if __name__ == "__main__":
    query = '"vitamin" AND PUB_YEAR:2025'

    process_topic(query, "vitamin", persist_dir=PERSIST_DIR, batch_size=5)

    print("=== 🎉 모든 작업 완료 ===")


🔎 vitamin: EuropePMC 검색 결과 개수: 25348개


vitamin 진행:   0%|          | 1/25348 [00:01<8:38:56,  1.23s/논문]

[1/25348] ⚠️ PMCID 없음 → 건너뜀


vitamin 진행:   0%|          | 2/25348 [00:02<8:33:29,  1.22s/논문]

[2/25348] ✅ 저장 완료 (PMCID: PMC12198872)
[3/25348] ✅ 저장 완료 (PMCID: PMC12166184)


...

vitamin 진행:   9%|▉         | 2254/25348 [46:12<7:10:23,  1.12s/논문]

[2254/25348] ✅ 저장 완료 (PMCID: PMC12074973)
[2255/25348] ⚠️ PMCID 없음 → 건너뜀

🛑 중단 감지! 지금까지의 데이터를 저장합니다...


vitamin 진행:   9%|▉         | 2255/25348 [46:14<7:53:33,  1.23s/논문]

⚡ 중단 시점까지 1229개 논문 저장 완료


KeyboardInterrupt: 